Alice Itzhak - 206820821

In [30]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, optimizers, initializers


(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()


In [31]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

validation_split = 0.2
split_index = int(len(x_train) * (1 - validation_split))
x_val, y_val = x_train[split_index:], y_train[split_index:]
x_train_new, y_train_new = x_train[:split_index], y_train[:split_index]

In [32]:
he_initializer = initializers.HeNormal()

model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    layers.BatchNormalization(),

    layers.Conv2D(
        20, (3, 3), padding='same',
        activation='leaky_relu',
        kernel_initializer=he_initializer,
        kernel_regularizer=regularizers.l2(0.002)
                  ),
    layers.BatchNormalization(),

    layers.Conv2D(
        68, (3, 3), padding='same',
        activation='leaky_relu',
        kernel_initializer=he_initializer,
        kernel_regularizer=regularizers.l2(0.002)
                   ),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Conv2D(
        68, (3, 3), padding='same',
        activation='leaky_relu',
        kernel_initializer=he_initializer,
        kernel_regularizer=regularizers.l2(0.002)
        ),
    layers.BatchNormalization(),
    layers.Dropout(0.35),

    layers.Conv2D(
        82, (3, 3), padding='same',
        activation='leaky_relu',
        kernel_initializer=he_initializer,
        kernel_regularizer=regularizers.l2(0.002)
        ),
    layers.BatchNormalization(),

    layers.Conv2D(
        82, (3, 3), padding='same',
        activation='leaky_relu',
        kernel_initializer=he_initializer,
        kernel_regularizer=regularizers.l2(0.002)
        ),
    layers.BatchNormalization(),
    layers.Dropout(0.4),

    layers.Flatten(),

    layers.Dense(
        21,
        activation='leaky_relu',
        kernel_initializer=he_initializer,
        kernel_regularizer=regularizers.l2(0.003)
        ),
    layers.BatchNormalization(),

    layers.Dense(
        68,
        activation='leaky_relu',
        kernel_initializer=he_initializer,
        kernel_regularizer=regularizers.l2(0.003)
        ),
    layers.BatchNormalization(),
    layers.Dropout(0.4),

    layers.Dense(num_classes, activation='softmax')
])


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=5e-4,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-7
    ),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
callbacks = [
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6

    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=6,
        restore_best_weights=True
    )
]

epochs = 30
batch_size = 32

history = model.fit(
    x_train_new, y_train_new,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(x_val, y_val),
    callbacks=callbacks
)


In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.figure(figsize=(12, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
